Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    Fill this in later.

In [11]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv')
#status column is not needed so that will be dropped

services.drop(labels=['status'], axis=1, inplace=True)
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services.astype({'date' : 'datetime64',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})

# testing stuff
njt_lines=services['line'].unique()

for line in njt_lines:
    print(line)

Northeast Corrdr
No Jersey Coast
Main Line
Morristown Line
Gladstone Branch
Raritan Valley
Bergen Co. Line 
Atl. City Line
Montclair-Boonton
Princeton Shuttle
Pascack Valley
